In [43]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import pickle
import re

In [4]:
file_name = 'genius_artist_and_top_songs_urls'
infile =open(file_name, 'rb')
genius_artist_and_top_songs_urls = pickle.load(infile)
infile.close()

In [79]:
genius_artist_and_top_songs_urls.head()

,artist,genius_artist_id,songs_list,page2,url_list1,url_list2
0,Cam'ron,1,<Response [200]>,<Response [200]>,"[https://genius.com/Kanye-west-gone-lyrics, ht...",[https://genius.com/Camron-what-means-the-worl...
1,JAY-Z,2,<Response [200]>,<Response [200]>,[https://genius.com/Beyonce-drunk-in-love-lyri...,[https://genius.com/Big-l-7-minute-freestyle-l...
2,Fabolous,3,<Response [200]>,<Response [200]>,[https://genius.com/Chris-brown-deuces-remix-l...,[https://genius.com/Fabolous-who-do-you-love-r...
3,Lil Wayne,4,<Response [200]>,<Response [200]>,[https://genius.com/Drake-in-my-feelings-lyric...,[https://genius.com/The-game-celebration-lyric...
4,Clipse,5,<Response [200]>,<Response [200]>,"[https://genius.com/Clipse-grindin-lyrics, htt...",[https://genius.com/Clipse-numb-it-down-lyrics...


In [99]:
def get_lyrics(soup_text):
    try:
        lyrics = soup_text.find_all("p")[0].text
        lyrics = re.sub(r'\[.*?\']','',lyrics)
        lyrics = re.sub(r'--','',lyrics)
        lyrics = lyrics.split("\n")
        lyrics = [ x for x in lyrics if x != ""]
        return lyrics
    except:
        np.nan
        
def get_links(x,i):
    return x[i]['href']
def get_table_row(side_list_table_row):
    a = side_list_table_row.find_all("span", {"class":"metadata_unit-label"})
    label = a[0].text
    links_list = side_list_table_row.find_all("a")
    links = [get_links(links_list,i) for i in range(len(links_list))]
    return label, links
def get_info_from_page(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.text,'html.parser')
    side_list_table = soup.find_all("div", {"class":"metadata_unit metadata_unit--table_row"} )
    lyrics = get_lyrics(soup)
    return  (url,[get_table_row(side_list_table[i]) for i in range(0,len(side_list_table)) ],lyrics )

In [90]:
l = get_info_from_page(genius_artist_and_top_songs_urls["url_list1"][0][0])

In [146]:
#dropping peskying NoneTypes. Can't replace with np.nan, cant iterate over floats.
artists = genius_artist_and_top_songs_urls.dropna()

In [175]:
url_list = [item for sublist in artists["url_list2"] for item in sublist]

In [177]:
len(url_list)

125432

In [176]:
#average number of links per artist.
len(url_list)/genius_artist_and_top_songs_urls.shape[0]

8.254277441431956

In [147]:
def log_progress(sequence, every=None, size=None, name='Items'):
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display

    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = int(size / 200)     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)

    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{name}: {index} / ?'.format(
                        name=name,
                        index=index
                    )
                else:
                    progress.value = index
                    label.value = u'{name}: {index} / {size}'.format(
                        name=name,
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = "{name}: {index}".format(
            name=name,
            index=str(index or '?')
        )

In [178]:
len(url_list)

125432

In [ ]:
for i in log_progress(range(5643+3588,len(url_list))):
    info_list.append(get_info_from_page(url_list[i]))


In [184]:
len(info_list)

305898

In [185]:
filename = "genius_side_tables_and_lyrics"
outfile = open(filename,'wb')
pickle.dump(info_list, outfile)
outfile.close()